In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'BeerAdvo-RateBeer'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.03226545,  0.10778032,  0.11899314, -0.10983982,  0.1798627 ,
         0.05171625, -0.07391304, -0.13592677,  0.01624714, -0.17345538],
       [ 0.10554622, -0.0194958 ,  0.05008403, -0.13378151, -0.16268908,
        -0.08521008, -0.15747899,  0.06436975,  0.11714286, -0.10420168],
       [-0.01763628,  0.03504352, -0.12895098,  0.15849748,  0.14200641,
        -0.04397618,  0.14567109, -0.19125057,  0.12368301,  0.01328447],
       [-0.08078669, -0.14447806,  0.10544629, -0.03676248,  0.03373676,
        -0.14432678,  0.10121029,  0.1193646 ,  0.13373676, -0.10015129],
       [ 0.13848957, -0.09840049, -0.0386718 ,  0.13686981,  0.14193157,
         0.0447459 , -0.17047985, -0.03219275, -0.17392185, -0.02429642]]), array([[ 0.14641684,  0.1089434 , -0.10411435,  0.09793317, -0.04887   ,
        -0.15472281,  0.1858219 ,  0.00772648,  0.11029554,  0.0351555 ],
       [ 0.07213977, -0.07608491, -0.11516062, -0.0093932 ,  0.11628781,
        -0.10182228,  0.11516062, -0.05673

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.5
K: 1, L: 2, Precision:1.0
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.9285714285714286
K: 2, L: 2, Precision:0.9285714285714286
K: 2, L: 3, Precision:1.0
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.42857142857142855
K: 3, L: 2, Precision:0.8571428571428571
K: 3, L: 3, Precision:1.0
K: 3, L: 4, Precision:0.9285714285714286
K: 3, L: 5, Precision:1.0
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.5714285714285714
K: 4, L: 2, Precision:0.8571428571428571
K: 4, L: 3, Precision:0.7857142857142857
K: 4, L: 4, Precis

K: 23, L: 4, Precision:0.0
K: 23, L: 5, Precision:0.0
K: 23, L: 6, Precision:0.21428571428571427
K: 23, L: 7, Precision:0.2857142857142857
K: 23, L: 8, Precision:0.07142857142857142
K: 23, L: 9, Precision:0.07142857142857142
K: 23, L: 10, Precision:0.14285714285714285
K: 24, L: 1, Precision:0.0
K: 24, L: 2, Precision:0.0
K: 24, L: 3, Precision:0.07142857142857142
K: 24, L: 4, Precision:0.0
K: 24, L: 5, Precision:0.0
K: 24, L: 6, Precision:0.0
K: 24, L: 7, Precision:0.0
K: 24, L: 8, Precision:0.07142857142857142
K: 24, L: 9, Precision:0.07142857142857142
K: 24, L: 10, Precision:0.07142857142857142
K: 25, L: 1, Precision:0.0
K: 25, L: 2, Precision:0.0
K: 25, L: 3, Precision:0.0
K: 25, L: 4, Precision:0.0
K: 25, L: 5, Precision:0.0
K: 25, L: 6, Precision:0.21428571428571427
K: 25, L: 7, Precision:0.0
K: 25, L: 8, Precision:0.07142857142857142
K: 25, L: 9, Precision:0.07142857142857142
K: 25, L: 10, Precision:0.07142857142857142
K: 26, L: 1, Precision:0.0
K: 26, L: 2, Precision:0.071428571